In [12]:
from client import DiffbotClient,DiffbotCrawl
#from config import API_TOKEN   #file doesn not exist
import pprint
import time
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models, similarities
from itertools import chain
from collections import Counter

class MyPrettyPrinter(pprint.PrettyPrinter):
    def format(self, object, context, maxlevels, level):
        if isinstance(object, unicode):
            return (object.encode('ascii'), True, False)
        return pprint.PrettyPrinter.format(self, object, context, maxlevels, level)

api = "article"
token = "d40071eafe3464dd1699f98240a22f10"
#print ("Calling article API endpoint on the url: http://shichuan.github.io/javascript-patterns/...\n")
diffbot = DiffbotClient()

C:\Users\Arghyadeep\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
urls = ["https://en.wikipedia.org/wiki/Biryani",
       "https://en.wikipedia.org/wiki/Lasagne",
       "https://en.wikipedia.org/wiki/Shawarma",
       "https://en.wikipedia.org/wiki/Hamburger",    #####food
       "https://en.wikipedia.org/wiki/Noodle",
       "https://en.wikipedia.org/wiki/Coq_au_vin",
       "https://en.wikipedia.org/wiki/Risotto",
       "https://en.wikipedia.org/wiki/Greek_salad",
       
       "https://en.wikipedia.org/wiki/Michael_Jackson",
       "https://en.wikipedia.org/wiki/Jimi_Hendrix",
       "https://en.wikipedia.org/wiki/Brad_Pitt",
       "https://en.wikipedia.org/wiki/Shah_Rukh_Khan",
       "https://en.wikipedia.org/wiki/Cristiano_Ronaldo",    ####entertainment
       "https://en.wikipedia.org/wiki/Rowan_Atkinson",
       "https://en.wikipedia.org/wiki/A._R._Rahman",
       
       "https://en.wikipedia.org/wiki/NASA",
       "https://en.wikipedia.org/wiki/Indian_Space_Research_Organisation",
        "https://en.wikipedia.org/wiki/Apple_Inc.",
        "https://en.wikipedia.org/wiki/Google",     ####technology
        "https://en.wikipedia.org/wiki/Tesla",
        "https://en.wikipedia.org/wiki/GlaxoSmithKline",
        "https://en.wikipedia.org/wiki/Audi",
        
       
       "https://en.wikipedia.org/wiki/Football",
       "https://en.wikipedia.org/wiki/Hockey",
       "https://en.wikipedia.org/wiki/Cricket",
       "https://en.wikipedia.org/wiki/Archery",      ####sports
       "https://en.wikipedia.org/wiki/Golf",
       "https://en.wikipedia.org/wiki/American_Football",
       "https://en.wikipedia.org/wiki/Tennis"]

In [10]:
#read from urls using diffbot api and save text data in text files
for i in range(len(urls)):
    filename = "out-00" + str(i) + ".txt"
    response = diffbot.request(urls[i],token,api)
    with open(filename, "w", encoding = "utf-8") as text_file:
        text_file.write(response['objects'][0]['text'])

In [21]:
#read from text files and get data in an array
texts = []
for i in range(len(urls)):
    filename = "out-00" + str(i) + ".txt"
    wrapper = open(filename, "r", encoding="utf-8")
    text = wrapper.readlines()
    texts.append(text)

In [56]:
#filter all texts with lemmatizer, removing numbers, lower all words
lemmatizer = WordNetLemmatizer()
all_words = set()
stop_words = list(stopwords.words('english'))

refined_texts = []
for docs in texts:
    documents = []
    for words in docs:
        temp = words.split()
        for word in temp:
            refine1 = word.lower()
            refine2 = lemmatizer.lemmatize(refine1)
            if refine2.isalpha() and refine2 not in stop_words:
                all_words.add(refine2)
                refine3 = refine2.lower()
                documents.append(refine3)
    refined_texts.append(' '.join(documents))



13287


In [73]:
#dictionary to count ocurrence of each word accross number of documents (inverse document frequency)
counts = {}
for word in all_words:
    for doc in refined_texts:
        if word in doc and word in counts:
            counts[word] += 1
        elif word in doc and word not in counts:
            counts[word] = 1
c = Counter(counts)


In [171]:
#remove words with high frequency ocurring accross multiple documents and also remove words ocurring in very few documents
refined_words = []
for word in dict(c):
    if dict(c)[word] < 15:
        refined_words.append(word)


In [172]:
#refine all documents with refined words
final_refined_texts = []
final_refined_texts_lists = []
for doc in refined_texts:
    temp = []
    for  word in doc.split():
        if word in refined_words:
            temp.append(word)
    final_refined_texts.append(" ".join(temp))
    final_refined_texts_lists.append(temp)


In [173]:
#create dictionary and corpus for lda model
dictionary = corpora.Dictionary(final_refined_texts_lists)
corpus = [dictionary.doc2bow(text) for text in final_refined_texts_lists]

In [174]:
#train lda model
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, iterations=1000, num_topics = 4)

In [175]:
#visualize model topics with word probabilities
for top in lda.print_topics():
    print(top)

(0, '0.008*"audi" + 0.007*"hamburger" + 0.007*"noodle" + 0.005*"space" + 0.004*"nasa" + 0.004*"burger" + 0.003*"rahman" + 0.003*"google" + 0.003*"bow" + 0.003*"meat"')
(1, '0.012*"football" + 0.010*"apple" + 0.008*"player" + 0.007*"audi" + 0.007*"game" + 0.006*"google" + 0.006*"space" + 0.006*"iphone" + 0.005*"goal" + 0.005*"ronaldo"')
(2, '0.011*"player" + 0.010*"biryani" + 0.009*"google" + 0.008*"tennis" + 0.006*"atkinson" + 0.004*"hendrix" + 0.004*"dish" + 0.003*"game" + 0.003*"bean" + 0.003*"tournament"')
(3, '0.008*"jackson" + 0.007*"apple" + 0.007*"hendrix" + 0.006*"film" + 0.006*"golf" + 0.005*"player" + 0.005*"iphone" + 0.005*"music" + 0.004*"pitt" + 0.004*"hockey"')


In [176]:
from gensim.models import Word2Vec

In [180]:
#define word2vec model
model = Word2Vec(final_refined_texts_lists)

In [212]:
##get the strings for the top 4 words for every topic
topics = []
threshold = 4
for top in lda.print_topics():
    topic_words = []
    for temp in str(top[1]).split("+"):
        topic_words.append(temp[7:-1].replace('"',''))
    topics.append(topic_words[:threshold])
    

In [213]:
topics

[['audi', 'hamburger', 'noodle', 'space'],
 ['football', 'apple', 'player', 'audi'],
 ['player', 'biryani', 'google', 'tennis'],
 ['jackson', 'apple', 'hendrix', 'film']]

In [231]:
#get the best tags for every topic
tags = []
for topic in topics:
    best_tag = ""
    model_similarity = 0
    for i in range(1,len(topic)):
        for j in range(i):
            #print(topic[i],topic[j])
            if model.similarity(topic[i],topic[j]) > model_similarity:
                model_similarity = model.similarity(topic[i],topic[j])
                best_tag = topic[i] + "," + topic[j]
    tags.append(best_tag)
            
print(tags)
        
        

['space,hamburger', 'audi,apple', 'tennis,player', 'hendrix,jackson']


C:\Users\Arghyadeep\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  
C:\Users\Arghyadeep\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  if __name__ == '__main__':


In [251]:
final_tags = []
for tag in tags:
    tag1 = str(tag.split(',')[0])
    tag2 = str(tag.split(',')[1])
    temp = [i[0] for i in model.most_similar(tag1)[:3]] + [j[0] for j in model.most_similar(tag2)[:3]]
    final_tags.append(temp)


C:\Users\Arghyadeep\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """


In [252]:
final_tags

[['nasa', 'mission', 'program', 'burger', 'chain', 'steak'],
 ['model', 'engine', 'quattro', 'announced', 'september', 'introduced'],
 ['tournament', 'player', 'playing', 'match', 'score', 'game'],
 ['guitar', 'experience', 'performance', 'michael', 'music', 'song']]

In [254]:
from gensim.models import KeyedVectors
filename = 'GoogleNews-vectors-negative300.bin'
loaded_model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [257]:
tags = []
for topic in topics:
    best_tag = ""
    model_similarity = 0
    for i in range(1,len(topic)):
        for j in range(i):
            #print(topic[i],topic[j])
            if topic[i] in loaded_model and topic[j] in loaded_model and loaded_model.similarity(topic[i],topic[j]) > model_similarity:
                model_similarity = loaded_model.similarity(topic[i],topic[j])
                best_tag = topic[i] + "," + topic[j]
    tags.append(best_tag)
            
print(tags)

['noodle,hamburger', 'player,football', 'tennis,player', 'apple,jackson']


In [258]:
new_final_tags = []
for tag in tags:
    tag1 = str(tag.split(',')[0])
    tag2 = str(tag.split(',')[1])
    temp = [i[0] for i in loaded_model.most_similar(tag1)[:3]] + [j[0] for j in model.most_similar(tag2)[:3]]
    new_final_tags.append(temp)

C:\Users\Arghyadeep\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """


In [259]:
new_final_tags

[['noodles', 'ramen', 'soba', 'burger', 'chain', 'steak'],
 ['players', 'playmaker', 'Player', 'rugby', 'game', 'code'],
 ['Tennis', 'volleyball', 'badminton', 'match', 'score', 'game'],
 ['apples', 'pear', 'fruit', 'michael', 'music', 'song']]